In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os

In [4]:
def int_trans(digit):
     x = filter(str.isdigit,digit)
     x=float("".join(list(x)))
     x=x/100000000
     x=round(x,2)
     return x

In [5]:
country_name=pd.read_excel(r'D:\YQ-SASS\上海海关数据更新\data\country_name.xlsx')
#去除字符串中的空格
country_name['中文名称']=country_name['中文名称'].str.replace(' ','')
country_name_map=dict(zip(country_name['中文名称'],country_name['大洲']))

In [7]:
current_path = os.getcwd()
parent_path = os.path.abspath(os.path.dirname(os.getcwd()))

In [10]:
parent_path + '\data\上海重点进出口商品/'

'd:\\YQ-SASS\\上海海关数据更新\\data\\上海重点进出口商品'

In [16]:
update_month  = '5月'

In [29]:
def important_product(parent_path):
    for product in['集成电路','笔记本电脑','锂离子蓄电池','电动载人汽车'] :
        if product == '集成电路':
            eximlist=['进口','出口']
        else:
            eximlist=['出口']
        for exim in eximlist:
            df_product=pd.DataFrame()
            for year in ['2019','2022','2023','2024']:
                df=pd.read_csv(parent_path + '\data\上海重点进出口商品/'+r'%s%s%s.csv'%(year,product,exim),encoding='gbk')
                df['人民币']=df['人民币'].apply(lambda x:int_trans(x))
                df_pt=pd.pivot_table(df[['贸易伙伴名称','人民币']],index='贸易伙伴名称',values='人民币',aggfunc=np.sum).reset_index()
                df_pt['年份']=year
                df_product=pd.concat([df_pt,df_product],axis=0,ignore_index=True)

            print(product)   
            df_2019=df_product[df_product['年份']=='2019']
            df_2022=df_product[df_product['年份']=='2022']
            df_2023=df_product[df_product['年份']=='2023']
            df_2024=df_product[df_product['年份']=='2024']
            
            df_2019['2019年占比']=df_2019['人民币']/df_2019['人民币'].sum()
            df_2022['2022年占比']=df_2022['人民币']/df_2022['人民币'].sum()
            df_2023['2023年1-11月占比']=df_2023['人民币']/df_2023['人民币'].sum()
            df_2024[f'2024年1-{update_month}占比']=df_2024['人民币']/df_2024['人民币'].sum()
            
            df_2019.rename(columns={'人民币':'2019年金额'},inplace=True)
            df_2022.rename(columns={'人民币':'2022年金额'},inplace=True)
            df_2023.rename(columns={'人民币':'2023年1-11月金额'},inplace=True)
            df_2024.rename(columns={'人民币':f'2024年1-{update_month}金额'},inplace=True)
            
            res=df_2019.merge(df_2022,on='贸易伙伴名称',how='outer')
            res=res.merge(df_2023,on='贸易伙伴名称',how='outer')
            res=res.merge(df_2024,on='贸易伙伴名称',how='outer',suffixes=['_dd','_ss'])
            res=res[['贸易伙伴名称', '2019年金额', '2019年占比', '2022年金额', '2022年占比','2023年1-11月占比',f'2024年1-{update_month}金额',f'2024年1-{update_month}占比']]
            res=res.fillna(0)
            res.sort_values(by='2023年1-11月占比',ascending=False,inplace=True)
            res['大洲']=res['贸易伙伴名称'].map(country_name_map)
        
            res.to_excel(r'../output/%s%s.xlsx'%(product,exim),index=False)

        df_2023_= pd.read_csv(parent_path + f'\data\上海重点进出口商品/'+f'2023年1-{update_month}电动载人汽车出口.csv',encoding='gbk')
        df_2023_['人民币']=df_2023_['人民币'].apply(lambda x:int_trans(x))
        df_pt=pd.pivot_table(df_2023_[['贸易伙伴名称','人民币']],index='贸易伙伴名称',values='人民币',aggfunc=np.sum).reset_index()
        df_pt[f'2023年1-{update_month}占比']=df_pt['人民币']/df_pt['人民币'].sum()
        df_pt.merge(df_2024,on='贸易伙伴名称',how='outer',).to_excel(f'..//output/2023与2024年1到{update_month}.xlsx',index=False)



        
   

    
   
        
        